## IMPLEMENTATION OF SARIMA MODEL

In [ ]:
# Hide Warnings
import warnings
warnings.filterwarnings('ignore')

### Libraries and Modules

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.ticker as ticker

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

### Load data and set frequency

In [ ]:
# Load the original data
processed_data = pd.read_excel('../Data/df_time_series.xlsx')

In [ ]:
# Convert column 'DATE' to datetime type and set it as index
processed_data['FECHA'] = pd.to_datetime(processed_data['FECHA'])
# Set the index
processed_data.set_index('FECHA', inplace=True)
# Create a date range with a frequency of 15 days and set it as an index
processed_data = processed_data.asfreq('15D')
# Check the index with the set frequency
print(processed_data.index)

### Plot Distribution for each principle active

In [ ]:
# Iterar sobre cada serie en el DataFrame processed_data
for serie in processed_data.columns:
     if serie != 'Año':
         if serie == 'CLORURO DE SODIO' or serie == 'LACTATO DE SODIO':
             ylabel = 'Cantidad (L)'
         else:
             ylabel = 'Cantidad (G)'
         plt.figure(figsize=(16, 5))
         plt.plot(processed_data[serie], label=serie, linewidth=3, linestyle='solid')
         plt.grid(True, linestyle='--', alpha=0.8)
         plt.legend(fontsize=13)
         plt.xlabel('Meses', fontsize=13)
         plt.ylabel(ylabel, fontsize=13)
         plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: format(int(x), ',')))
         plt.title(f'Distribución de la serie de {serie}'.upper(), fontsize=15)
         plt.show()

### Augmented Dickey-Fuller Test for Seasonality (first difference)

In [ ]:
# Augmented Dickey-Fuller Test for Seasonality (first difference)
for column in processed_data:
    result = adfuller(processed_data[column].diff().dropna())
    print(f'ADF Statistic {column}: {result[0]}')
    print(f'p-value: {result[1]}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value} \n')

### Consumption top five groups with seasonality fixed first difference

In [ ]:
# Consumption top five groups with seasonality fixed first difference
for serie in processed_data.columns:
     if serie != 'Año':
        if serie == 'CLORURO DE SODIO' or serie == 'LACTATO DE SODIO':
             ylabel = 'Cantidad (L)'
        else:
             ylabel = 'Cantidad (G)'
        plt.figure(figsize=(16, 5))
        plt.plot(processed_data[serie].diff().dropna(), label=serie, linewidth=3, linestyle='solid')
        plt.grid(True, linestyle='--', alpha=0.8)
        plt.legend(fontsize=13)
        plt.xlabel('Meses', fontsize=13)
        plt.ylabel(ylabel, fontsize=13)
        plt.title(f'Distribución de la serie de {serie}'.upper(), fontsize=15)
        plt.show()

### Analysis of autocorrelation function (ACF) and partial autocorrelation function (PACF) first diff

In [ ]:
# Analysis of autocorrelation function (ACF) and partial autocorrelation function (PACF) first diff
for column in processed_data:
    if column != 'Año':
        if column not in ['Año', 'LACTATO DE SODIO', 'PROPOFOL']:
            # Aplicar diferenciación solo si la columna no es LACTATO DE SODIO o PROPOFOL
            consumption_diff = processed_data[column].diff().dropna()
            data_to_plot = consumption_diff
        else:
            # Si la columna es LACTATO DE SODIO o PROPOFOL, usar datos originales
            data_to_plot = processed_data[column]
        # Crear una figura con dos subgráficos
        fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    
        # Graficar la función de autocorrelación (ACF)
        plot_acf(consumption_diff, ax=ax[0], lags=20)
        ax[0].set_title(f'ACF - {column}')
    
        # Graficar la función de autocorrelación parcial (PACF)
        plot_pacf(consumption_diff, ax=ax[1], lags=20)
        ax[1].set_title(f'PACF - {column}')
    
        plt.show()

### Define SARIMA parameters for all active principles

In [ ]:
# Define SARIMA parameters for all active principles
sarima_params = {
    'CLORURO DE SODIO': {'order': (2, 1, 4), 'seasonal_order': (0, 1, 1, 15)},
    'LACTATO DE SODIO': {'order': (3, 0, 4), 'seasonal_order': (0, 0, 1, 15)},
    'METAMIZOL': {'order': (3, 1, 3), 'seasonal_order': (0, 1, 1, 15)},
    'PARACETAMOL': {'order': (3, 1, 4), 'seasonal_order': (0, 1, 1, 15)},
    'PROPOFOL': {'order': (2, 0, 3), 'seasonal_order': (0, 0, 1, 15)}
}

### Fit SARIMA models for each principle active time series, residuals distribution plotting and obtain metrics

In [ ]:
# Iterate over each active principle
for active_principle, params in sarima_params.items():
    # Get data for the current active principle
    x = processed_data[active_principle]
    # Split data into train and test sets with a 90-10 ratio
    train, test = train_test_split(x, test_size=0.1, shuffle=False)
    
    # Fit SARIMA
    model = SARIMAX(train, order=params['order'], seasonal_order=params['seasonal_order'])
    model_fit = model.fit()

    # Predictions for training RMSE
    train_predictions = model_fit.predict(start=train.index[0], end=train.index[-1])
    
    forecast = model_fit.get_forecast(steps=len(test), alpha=0.05)
    forecast_values = forecast.predicted_mean
    conf = forecast.conf_int()
    
    # Clip forecast values below 0
    forecast_values = np.maximum(forecast_values, 0)
    
    # Model summary
    print(model_fit.summary())

    # RMSE Train and Test
    rmse_train = np.sqrt(mean_squared_error(train, train_predictions))
    mse_train = mean_squared_error(train, train_predictions)
    mae_train = mean_absolute_error(train, train_predictions)

    rmse_test = np.sqrt(mean_squared_error(test, forecast_values))
    
    print(f"For {active_principle}:")
    print(f"Training RMSE: {rmse_train}")
    print(f"Training MSE: {mse_train}")
    print(f"Training MAE: {mae_train}")
    print(f"Test RMSE: {rmse_test}")
    
    # Plot residuals
    residualsSar = {}
    n_rows = 1
    n_cols = 2
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    column = active_principle
    residualsSar[column] = model_fit.resid
    axes[0].plot(residualsSar[column], label=f'residuos')
    axes[0].set_title(f'Residuos para {column}'.upper())
    axes[0].set_xlabel('Rezagos')
    axes[0].set_ylabel('Valor Residuo')
    axes[0].legend()
    sns.kdeplot(residualsSar[column], label=f'{column} Residuals', fill=True, ax=axes[1])
    axes[1].set_title(f'Densidad de residuos para {column}'.upper())
    axes[1].set_xlabel('Valor Residuo')
    axes[1].set_ylabel('Densidad')
    axes[1].legend()
    plt.tight_layout()
    plt.show()
    
    # Forecast data
    # Last train data
    last_train_date = train.index[-1]
    
    # Biweekly frequency
    next_date = pd.to_datetime(last_train_date) + pd.Timedelta(days=15)
    
    # Generate dates for forecast data
    forecast_dates = pd.date_range(start=next_date, periods=10, freq='15D').strftime('%Y-%m-%d').tolist()

    forecast_data = {
        'Date': forecast_dates,
        'FC_VALUES': forecast_values,
        'LOWER_VALUES': conf[f'lower {active_principle}'],
        'UPPER_VALUES': conf[f'upper {active_principle}']
    }

    # Create forecast DataFrame
    df_forecast = pd.DataFrame(forecast_data)
    print(df_forecast)

    # Set index for data series visualization
    df_forecast['Date'] = pd.to_datetime(df_forecast['Date'])
    df_forecast.set_index('Date', inplace=True)

    # Plot
    plt.figure(figsize=(12, 5))
    plt.plot(train, label='Entrenamiento')
    plt.plot(test, label='Prueba')
    plt.plot(df_forecast['FC_VALUES'], label='Pronósticos', color='green')
    plt.fill_between(df_forecast.index, df_forecast['LOWER_VALUES'], df_forecast['UPPER_VALUES'], color='k', alpha=0.2)
    plt.title(f'Pronósticos serie {active_principle}'.upper())
    plt.legend(loc='upper left', fontsize=10)
    plt.show()

    # Calculate metrics
    mse = mean_squared_error(test, forecast_values)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test, forecast_values)
    
    print(f"For {active_principle}:")
    print(f"RMSE: {rmse}")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")

### Future Forecast for next five months for each time series

In [ ]:
# Iterate over each active principle
for active_principle, params in sarima_params.items():
    # Get data for the current active principle
    x = processed_data[active_principle]
    # Split data into train and test sets with an 90-10 ratio
    train, test = train_test_split(x, test_size=0.1, shuffle=False)
    
    # Fit SARIMA
    model = SARIMAX(train, order=params['order'], seasonal_order=params['seasonal_order'])
    model_fit = model.fit()
    
    # Make forecast for one five months
    forecast = model_fit.get_forecast(steps=10, alpha=0.05)
    forecast_values = forecast.predicted_mean
    conf = forecast.conf_int()
    
    # Forecast five months
    forecast_dates = pd.date_range(start=x.index[-1] + pd.Timedelta(days=15), periods=10, freq='15D')
    
    # Create forecast DataFrame
    forecast_data = {
        'Date': forecast_dates,
        'FC_VALUES': forecast_values,
        'LOWER_VALUES': conf[f'lower {active_principle}'],
        'UPPER_VALUES': conf[f'upper {active_principle}']
    }
    df_forecast = pd.DataFrame(forecast_data)
    
    y_label = 'Cantidad (L)' if active_principle == 'CLORURO DE SODIO' or active_principle == 'LACTATO DE SODIO' else 'Cantidad (G)'

    # Plot forecast
    plt.figure(figsize=(12, 5))
    plt.plot(train, label='Entrenamiento')
    plt.plot(test, label='Prueba')
    plt.plot(df_forecast['Date'], df_forecast['FC_VALUES'], label='Forecast', color='green')
    plt.fill_between(df_forecast['Date'], df_forecast['LOWER_VALUES'], df_forecast['UPPER_VALUES'], color='k', alpha=0.2)
    plt.title(f'Pronósticos para los siguientes 5 meses - {active_principle}'.upper())
    plt.xlabel('Años')
    plt.ylabel(y_label)
    plt.legend(loc='upper left', fontsize=10)
    plt.show()